<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Libraries

This example uses a foundational library called `icoscp_core` which can be used to access time-series ICOS data that are <i>previewable</i> in the ICOS Data Portal. "Previewable" means that it is possible to visualize the data variables in the preview plot. The library can also be used to access (meta-)data from [ICOS Cities](https://citydata.icos-cp.eu/portal/) and [SITES](https://data.fieldsites.se/portal/) data repositories. 

General information on all ICOS Carbon Portal Python libraries can be found on our [help pages](https://icos-carbon-portal.github.io/pylib/). 

Documentation of the `icoscp_core` library, including information on running it locally, can also be found on [PyPI.org](https://pypi.org/project/icoscp_core/).

Note that for running this example locally, authentication is required (see the `how_to_authenticate.ipynb` notebook).

# Example: Access metadata and data of a single data object


## Import the ICOS modules

In [ ]:
from icoscp_core.icos import meta, data

 ## Fetch full data object metadata

In [ ]:
landing_page_uri = 'https://meta.icos-cp.eu/objects/mcZCu-5WouAxMyUJ8RJZ9y5j'
dobj_meta = meta.get_dobj_meta(landing_page_uri)

## Access metadata properties
### Get help on data object metadata
For assistance with metadata-access code, one can use autocomplete with Tab key after "."

Alternatively, it is possible to discover the metadata structure and its substructures as follows:

In [ ]:
from icoscp_core import metacore
help(metacore.DataObject)
#help(metacore.References)
#help(metacore.StationTimeSeriesMeta)
#help(metacore.DataAcquisition)
#help(metacore.Station)

### List column metadata of a station-specific time-series data object

In [ ]:
dobj_meta.specificInfo.columns

### List column labels only

In [ ]:
[col.label for col in dobj_meta.specificInfo.columns]

### Textual citation string

In [ ]:
dobj_meta.references.citationString

### BibTeX citation string

In [ ]:
dobj_meta.references.citationBibTex

### Data licence metadata

In [ ]:
dobj_meta.references.licence

## Get the data

This step would require authentication for local use (non on an ICOS-hosted Jupyter service)

**NOTE** the following example is appropriate mostly for the case of *single* data object, or for inhomogeneous lists of data objects of different data types. For homogeneous lists, it is highly recommended to use `batch_get_columns_as_arrays` method for much better performance due to reduced number of server calls.

In [ ]:
import pandas as pd
dobj_cols = data.get_columns_as_arrays(dobj_meta)
dobj_data = pd.DataFrame(dobj_cols)

## Make a plot

In [ ]:
import matplotlib.pyplot as plt
dobj_data.plot(x='TIMESTAMP', y='ch4', grid=True)
plt.show()

## Make a better plot

In [ ]:
# get 'value type' part of column metadata for ch4 column
columns_meta = dobj_meta.specificInfo.columns
ch4_value_type = [col for col in columns_meta if col.label=='ch4'][0].valueType

y_axis_label = f"{ch4_value_type.self.label} [{ch4_value_type.unit}]"
station = dobj_meta.specificInfo.acquisition.station.org.name

plot = dobj_data.plot(x='TIMESTAMP', y='ch4', grid=True, title=station)
plot.set(ylabel=y_axis_label)
plt.show()

### Add monthly averages to the plot

In [ ]:
# ch4 timeseries: same as before
plot = dobj_data.plot(x='TIMESTAMP', y='ch4', grid=True, title=station)
plot.set(ylabel=y_axis_label)

# compute monthly averages
by_month = dobj_data.groupby(pd.Grouper(key='TIMESTAMP', freq='M')).mean().reset_index()

by_month.plot(x='TIMESTAMP', y='ch4', grid=True, ax=plot, color='red', linewidth=2, label='ch4 monthly')

plt.show()

## Make an interactive plot

In [ ]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, reset_output
reset_output()
output_notebook()

In [ ]:
x = list(dobj_data['TIMESTAMP'])
y = list(dobj_data['ch4'])

# create a new plot with a title and axis labels
p = figure(x_axis_label="TIMESTAMP", x_axis_type="datetime", y_axis_label=y_axis_label, width=1000)

# add a line renderer with legend and line thickness
p.line(x, y, legend_label=station, line_width=1)

# show the results
show(p)